In [1]:
from PIL import Image
import torch
import os
import numpy as np
import cv2
import random
from diffusers import FluxFillPipeline, FluxPriorReduxPipeline
import gc
from huggingface_hub import snapshot_download
from google.colab import userdata
from transformers import pipeline, AutoImageProcessor, AutoProcessor, AutoModelForZeroShotObjectDetection
from io import BytesIO

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [2]:
device = "cuda"
dtype = torch.bfloat16
im_w, im_h = (768, 768)
HF_TOKEN = userdata.get('HF_TOKEN')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
snapshot_download(repo_id="black-forest-labs/FLUX.1-Fill-dev", local_dir="./FLUX.1-Fill-dev", token=HF_TOKEN)
snapshot_download(repo_id="black-forest-labs/FLUX.1-Redux-dev", local_dir="./FLUX.1-Redux-dev", token=HF_TOKEN)
snapshot_download(repo_id="WensongSong/Insert-Anything", local_dir="./insert_anything_model", token=HF_TOKEN)

Fetching 28 files:   0%|          | 0/28 [00:00<?, ?it/s]

flux1-fill-dev.safetensors:   0%|          | 0.00/23.8G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/540 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/299 [00:00<?, ?B/s]

...

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

20250321_steps5000_pytorch_lora_weights.(…):   0%|          | 0.00/1.85G [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

'/content/insert_anything_model'

In [5]:
grounding_dino_model_id = "IDEA-Research/grounding-dino-tiny"
grounding_dino_processor = AutoProcessor.from_pretrained(grounding_dino_model_id) # for input prep
grounding_dino_model = AutoModelForZeroShotObjectDetection.from_pretrained(grounding_dino_model_id).to(device)

preprocessor_config.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/689M [00:00<?, ?B/s]

In [6]:
def get_bboxes(image, text_labels):

  inputs = grounding_dino_processor (
      images=image, text=text_labels, return_tensors="pt").to(device)

  with torch.no_grad():
      outputs = grounding_dino_model(**inputs)

  results = grounding_dino_processor.post_process_grounded_object_detection (
      outputs,
      inputs.input_ids,
      target_sizes = [image.size[::-1]]
  )

  # Extract the bounding boxes
  boxes  = results[0]["boxes"]
  scores = results[0]["scores"]
  labels = results[0]["labels"]

  return [boxes.tolist()], labels

In [34]:
source_image = Image.open("/content/drive/MyDrive/images/model-2.jpg").resize((im_w, im_h))
boxes_influencer, labels_influencer = get_bboxes (source_image, ["right hand."])

/usr/local/lib/python3.12/dist-packages/transformers/models/grounding_dino/processing_grounding_dino.py:93: FutureWarning: The key `labels` is will return integer ids in `GroundingDinoProcessor.post_process_grounded_object_detection` output since v4.51.0. Use `text_labels` instead to retrieve string object names.
  warnings.warn(self.message, FutureWarning)


In [35]:
print(boxes_influencer)

[[[101.73677062988281, 336.792724609375, 285.00299072265625, 524.169189453125]]]


In [36]:
print(labels_influencer)

['right hand']


In [10]:
ref_image = Image.open("/content/drive/MyDrive/images/can-of-coke.jpg").resize((im_w, im_h))
boxes_product, labels_product = get_bboxes (ref_image, ["red can."])

In [11]:
print(boxes_product)

[[[270.62408447265625, 231.16632080078125, 506.51556396484375, 638.0360107421875]]]


In [12]:
boxes_product[0][0]

[270.62408447265625, 231.16632080078125, 506.51556396484375, 638.0360107421875]

In [13]:
print(labels_product)

['red can']


#### Create the masks (product mask, influencer mask)

In [14]:
product_mask = np.ones((im_h, im_w, 3), dtype=np.uint8) * 255 # white

# xmin, ymin, xmax, ymax = boxes_product[0][0][0], boxes_product[0][0][1], boxes_product[0][0][2], boxes_product[0][0][3]
xmin, ymin, xmax, ymax = boxes_product[0][0]
x, y, w, h = int(xmin), int(ymin), int(xmax-xmin), int(ymax-ymin) # convert pixels to int
product_mask[y:y+h, x:x+w] = np.array(ref_image)[y:y+h, x:x+w]

In [15]:
Image.fromarray(product_mask).save("productmask.png")

In [37]:
h_concat_image = cv2.hconcat([product_mask, np.array(source_image)])
Image.fromarray(h_concat_image).save("hconcat.png")

In [38]:
influencer_mask = np.zeros((im_h, im_w * 2, 3), dtype=np.uint8)

xmin, ymin, xmax, ymax = boxes_influencer[0][0]
x, y, w, h = int(xmin), int(ymin), int(xmax-xmin), int(ymax-ymin)
influencer_mask[y:y+h, x+im_w:x+im_w+w] = 255

In [39]:
Image.fromarray(influencer_mask).save("influencermask.png")

# Testing

In [40]:
pipe = FluxFillPipeline.from_pretrained("./FLUX.1-Fill-dev", torch_dtype=dtype)

pipe.load_lora_weights(
    "./insert_anything_model/20250321_steps5000_pytorch_lora_weights.safetensors"
)

redux = FluxPriorReduxPipeline.from_pretrained("./FLUX.1-Redux-dev").to(dtype=dtype)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

In [41]:
pipe.enable_sequential_cpu_offload() # extreme VRAM saving, slow inference

In [42]:
pipe_prior_output = redux(Image.fromarray(product_mask))

In [43]:
n_outputs = 2

for i in range(n_outputs):
    random_seed = random.randint(0, 2**32 - 1)
    generator = torch.Generator(device).manual_seed(random_seed)
    edited_image = pipe(
        image = Image.fromarray(h_concat_image),
        mask_image = Image.fromarray(influencer_mask),
        height = im_h,
        width = im_w * 2,
        max_sequence_length = 512,
        generator = generator,
        **pipe_prior_output,
    ).images[0]

    edited_image.save(f"edited_image_{i}.png")

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [26]:
edited_image.size

(1536, 768)

In [45]:
im = cv2.imread("edited_image_0.png")[:, 768:, :]
# im_resized = cv2.resize(im, (768, 512))
# print(im_resized.shape)
cv2.imwrite("edited_image_0_final.png", im) # im_resized)

True

In [47]:
! cp /content/*.png /content/drive/MyDrive/results_final_2/